In [ ]:
# TEMPORARY: App Deployment Testing Notebook
# This notebook tests ONLY the app deployment part
# Can be deleted after testing is complete

import os, json, base64, requests, subprocess, shutil, platform
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

print("🔧 APP DEPLOYMENT TEST")
print("="*70)
print(f"👤 User: {user}")
print(f"🏢 Workspace: {api_url}")

In [ ]:
# Step 1: Upload App Files
print("\n" + "="*70)
print("📤 STEP 1: UPLOADING APP FILES")
print("="*70)

app_name = "waf-automation-tool"
workspace_path = f"/Users/{user}/waf-app-source"
app_source_dir = os.path.join(os.getcwd(), "streamlit-waf-automation")

# Create directory
mkdir_response = requests.post(
    url=f"{api_url}/api/2.0/workspace/mkdirs",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={"path": workspace_path}
)

# Upload each file
for item in os.listdir(app_source_dir):
    source = os.path.join(app_source_dir, item)
    if os.path.isfile(source):
        dest_path = f"{workspace_path}/{item}"
        
        with open(source, "rb") as f:
            content = f.read()
        
        upload_response = requests.post(
            url=f"{api_url}/api/2.0/workspace/import",
            headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
            json={
                "path": dest_path,
                "content": base64.b64encode(content).decode("utf-8"),
                "format": "AUTO",
                "language": "PYTHON" if item.endswith(".py") else "AUTO",
                "overwrite": True
            }
        )
        
        if upload_response.status_code in [200, 201]:
            print(f"   ✅ Uploaded: {item}")
        else:
            print(f"   ⚠️  Failed to upload {item}: {upload_response.status_code}")

print(f"\n✅ App files uploaded to {workspace_path}")

In [ ]:
# Step 2: Note about CLI (not supported in notebooks)
print("\n" + "="*70)
print("📝 STEP 2: DEPLOYMENT METHOD")
print("="*70)

print(f"   ℹ️  Note: Databricks CLI is not supported in notebook environments")
print(f"   ℹ️  Using REST API directly for app deployment")
print(f"   ℹ️  This is the recommended approach for notebooks")

In [ ]:
# Step 3: Deploy App using REST API (CLI not supported in notebooks)
print("\n" + "="*70)
print("🚀 STEP 3: DEPLOYING APP")
print("="*70)

app_url = None

print(f"   📁 Source path: /Workspace{workspace_path}")
print(f"   🔐 Using REST API (CLI not supported in notebooks)")

# Try to create or get app first
app_exists = False
try:
    get_app_response = requests.get(
        url=f"{api_url}/api/2.0/apps/{app_name}",
        headers={"Authorization": f"Bearer {token}"}
    )
    
    if get_app_response.status_code == 200:
        app_exists = True
        print(f"   ✅ App '{app_name}' already exists")
    elif get_app_response.status_code == 404:
        print(f"   📝 App '{app_name}' does not exist - will create")
    else:
        print(f"   ⚠️  Could not check app status: {get_app_response.status_code}")
except Exception as e:
    print(f"   ⚠️  Error checking app: {e}")

# Create app if it doesn't exist
if not app_exists:
    try:
        create_app_response = requests.post(
            url=f"{api_url}/api/2.0/apps",
            headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
            json={
                "name": app_name,
                "source_code_path": f"/Workspace{workspace_path}"
            }
        )
        
        if create_app_response.status_code in [200, 201]:
            print(f"   ✅ App '{app_name}' created")
        else:
            print(f"   ⚠️  Could not create app: {create_app_response.status_code}")
            print(f"      Response: {create_app_response.text[:200]}")
    except Exception as e:
        print(f"   ⚠️  Error creating app: {e}")

# Deploy app (create deployment)
try:
    deploy_response = requests.post(
        url=f"{api_url}/api/2.0/apps/{app_name}/deployments",
        headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
        json={
            "source_code_path": f"/Workspace{workspace_path}"
        }
    )
    
    if deploy_response.status_code in [200, 201]:
        result = deploy_response.json()
        deployment_id = result.get("deployment_id", "")
        print(f"\n✅ App deployed successfully via REST API")
        if deployment_id:
            print(f"   🆔 Deployment ID: {deployment_id}")
        
        # Get app info to retrieve URL
        get_app_response = requests.get(
            url=f"{api_url}/api/2.0/apps/{app_name}",
            headers={"Authorization": f"Bearer {token}"}
        )
        
        if get_app_response.status_code == 200:
            app_info = get_app_response.json()
            app_url = app_info.get("url", "")
            if app_url:
                print(f"   🚀 App URL: {app_url}")
    else:
        print(f"\n❌ REST API deployment failed: {deploy_response.status_code}")
        print(f"   Response: {deploy_response.text}")
        print(f"\n   💡 Note: Apps API may require different endpoint or parameters")
        print(f"   📝 Manual deployment recommended via UI")
except Exception as e:
    print(f"\n❌ Error deploying app via REST API: {e}")
    print(f"   📝 Manual deployment recommended")

# Always show manual steps
print(f"\n📝 Manual Deployment Steps (if API failed):")
print(f"   1. Go to Databricks Apps: {api_url}/apps")
print(f"   2. Click 'Create App' or find existing app '{app_name}'")
print(f"   3. Source code path: /Workspace{workspace_path}")
print(f"   4. Click 'Deploy'")

In [ ]:
# Summary
print("\n" + "="*70)
print("✅ APP DEPLOYMENT TEST COMPLETE")
print("="*70)
print(f"\n📊 Results:")
print(f"   App name: {app_name}")
print(f"   Source path: /Workspace{workspace_path}")
print(f"   Deployment method: REST API (CLI not supported in notebooks)")
if 'app_url' in locals() and app_url:
    print(f"   🚀 App URL: {app_url}")
else:
    print(f"   📝 App URL: Check manually at {api_url}/apps")
print("\n")